In [1]:
import os
import sys
import logging
import numpy as np
import pandas as pd
import urlparse
os.sys.path.append("/home/sparrow/sparrow/malicious_url_cluster")
from config import cfg
from logger import logger

cfg.N_CLUSTER_RATION = 200

cfg.EDIT_DISTANCE_THRESH_LONG = 0.3
cfg.EDIT_DISTANCE_THRESH_SHORT = 0.3
cfg.LONG_URL_THRESH = 100
cfg.SHORT_URL_THRESH = 4

cfg.MIN_SUBSTRING_SIZE = 4
cfg.SINGLE_REGEX_SIZE = 4
cfg.TOTAL_REGEX_SIZE = 4
cfg.SINGLE_REGEX_SIZE_RATIO = 0.2
cfg.TOTAL_REGEX_SIZE_RATIO = 0.2

cfg.DOMAIN_CLUSTER_SIZE_THRESH = 3

import preprocess
import vectorize
import clustering
import regex_url
import urlnormalize
import regex_domain
from file_io import load_urls, dump_urls
import url_forest
reload(url_forest)
import joblib as jl

## Prepare Testing Set

In [2]:
def get_malware_edr_data(file_list):
    root_path = "/home/sparrow/sparrow/data/sangfor/18"
    df_list = []
    for i in file_list:
        local_path = root_path + i
        filepath = os.path.join(local_path ,'dumped_result.edr.csv')
        df_list.append( pd.read_csv(filepath))
        print local_path
        
    df = pd.concat(df_list, axis = 0)
    return df

file_list = ["0401", "0402", "0410", "0411", "0412", "0415", "0416", "0417", "0418", "0419"]
df = get_malware_edr_data(file_list)
df = df.drop_duplicates("url")
urls = list(df.url)
print "\nEDR total count: %d\n" % len(df)

url_0 = list(df.loc[df.attr == 0].url)
url_1 = list(df.loc[df.attr == 1].url)
url_2 = list(df.loc[df.attr == 2].url)
url_3 = list(df.loc[df.attr == 3].url)
print "0 malware\t%d\n1 benign\t%d\n2 unknown\t%d\n3 possible benign\t%d" %(len(url_0), len(url_1), len(url_2), len(url_3))
df = df.set_index("url")

/home/sparrow/sparrow/data/sangfor/180401
/home/sparrow/sparrow/data/sangfor/180402
/home/sparrow/sparrow/data/sangfor/180410
/home/sparrow/sparrow/data/sangfor/180411
/home/sparrow/sparrow/data/sangfor/180412
/home/sparrow/sparrow/data/sangfor/180415
/home/sparrow/sparrow/data/sangfor/180416
/home/sparrow/sparrow/data/sangfor/180417
/home/sparrow/sparrow/data/sangfor/180418
/home/sparrow/sparrow/data/sangfor/180419

EDR total count: 36073

0 malware	2801
1 benign	296
2 unknown	32972
3 possible benign	4


## Prepare Trainning Data

In [9]:
sangfor_url1 = list(set(load_urls("../../data/sangfor/malware_0402.csv", csv = True)))
sangfor_url2 = list(set(load_urls("../../data/sangfor/malware_0410_0416.csv", csv = True)))
vt_url = list(set(load_urls("../../data/VirusTotalnew/malicious/180424-all.csv", csv = True)))
train_urls = list(set(sangfor_url1 + sangfor_url2 + vt_url) )
train_urls = [_ for _ in train_urls if isinstance(_, str)]
print len(train_urls)

2018-04-24 21:16:37 file_io.py [line:52] DEBUG	URLs Count:	2786
2018-04-24 21:16:37 file_io.py [line:52] DEBUG	URLs Count:	1571
2018-04-24 21:16:39 file_io.py [line:52] DEBUG	URLs Count:	1300012


1303296


In [10]:
dump_urls(train_urls, "../../data/EXP_domain/malware.csv")

2018-04-24 21:16:54 file_io.py [line:24] DEBUG	OLD DATA FIND! REMOVING	../../data/EXP_domain/malware.csv
2018-04-24 21:16:57 file_io.py [line:33] DEBUG	URLs has been dump	../../data/EXP_domain/malware.csv


In [11]:
url_domain, url_path = preprocess.make_url_split(train_urls)
print len(url_domain)
print len(url_path)
urls = list(url_domain)

453097
638153


In [12]:
cc_list = clustering.make_kmeans_cluster_mass(urls,domain = True, path = False, param = False)

2018-04-24 21:17:57 vectorize.py [line:77] DEBUG	vectorization complete! data shape:	(453097, 128)
2018-04-24 21:17:57 clustering.py [line:224] DEBUG	Begin to make k-means cluster, data_size: 453097 n_clusters: 226
2018-04-24 21:17:57 clustering.py [line:245] DEBUG	optimal n_cluster	9
2018-04-24 21:19:00 clustering.py [line:263] DEBUG	k-means cluster done!
2018-04-24 21:19:00 clustering.py [line:291] DEBUG	Preliminary K-means clustering complete
2018-04-24 21:19:07 vectorize.py [line:77] DEBUG	vectorization complete! data shape:	(118856, 128)
2018-04-24 21:19:07 clustering.py [line:224] DEBUG	Begin to make k-means cluster, data_size: 118856 n_clusters: 59
2018-04-24 21:19:07 clustering.py [line:245] DEBUG	optimal n_cluster	3
2018-04-24 21:19:33 clustering.py [line:263] DEBUG	k-means cluster done!
2018-04-24 21:19:37 vectorize.py [line:77] DEBUG	vectorization complete! data shape:	(68613, 128)
2018-04-24 21:19:37 clustering.py [line:224] DEBUG	Begin to make k-means cluster, data_size: 6

In [ ]:
clustering.make_string_distance_cluster(data = cc_list, 
                            metric = "distance",  
                            file_path = "../../data/EXP_domain/cluster_distance.json", )

batch 0 0/42783
batch 1 0/38329
batch 2 0/37802
batch 3 0/34109
batch 4 0/30976
batch 5 0/29751
batch 6 0/29631
batch 7 0/24401
batch 6 100/29631
batch 7 100/24401
batch 2 100/37802
batch 6 300/29631
batch 5 100/29751
batch 7 200/24401
batch 4 100/30976
batch 0 100/42783
batch 1 100/38329
batch 7 300/24401
batch 6 500/29631
batch 2 200/37802
batch 5 200/29751
batch 3 100/34109
batch 4 200/30976
batch 0 200/42783
batch 5 300/29751
batch 2 300/37802
batch 6 900/29631
batch 1 200/38329
batch 4 300/30976
batch 7 700/24401
batch 7 800/24401
batch 6 1200/29631
batch 2 400/37802
batch 0 300/42783
batch 3 200/34109
batch 7 900/24401
batch 5 500/29751
batch 6 1400/29631
batch 4 400/30976
batch 1 300/38329
batch 2 500/37802
batch 6 1600/29631
batch 7 1100/24401
batch 6 1700/29631
batch 5 600/29751
batch 0 400/42783
batch 7 1200/24401
batch 4 500/30976
batch 3 300/34109
batch 2 600/37802
batch 7 1300/24401
batch 5 700/29751
batch 1 400/38329
batch 7 1500/24401
batch 2 700/37802
batch 4 600/30976


batch 5 7800/29751
batch 1 4400/38329
batch 2 5400/37802
batch 5 7900/29751
batch 4 5000/30976
batch 8 3800/24138
batch 1 4500/38329
batch 2 5500/37802
batch 4 5100/30976
batch 8 4200/24138
batch 1 4600/38329
batch 0 4200/42783
batch 2 5600/37802
batch 3 4000/34109
batch 4 5200/30976
batch 1 4700/38329
batch 8 4600/24138
batch 5 8100/29751
batch 8 4700/24138
batch 2 5700/37802
batch 0 4300/42783
batch 8 4900/24138
batch 4 5300/30976
batch 1 4800/38329
batch 8 5000/24138
batch 5 8200/29751
batch 8 5100/24138
batch 4 5400/30976
batch 5 8300/29751
batch 5 8400/29751
batch 3 4200/34109
batch 8 5400/24138
batch 4 5500/30976
batch 8 5600/24138
batch 0 4500/42783
batch 8 5700/24138
batch 1 5100/38329
batch 2 6000/37802
batch 8 6000/24138
batch 4 5700/30976
batch 5 8800/29751
batch 2 6100/37802
batch 3 4300/34109
batch 4 5800/30976
batch 2 6200/37802
batch 5 9000/29751
batch 8 6800/24138
batch 1 5400/38329
batch 2 6300/37802
batch 4 5900/30976
batch 8 7200/24138
batch 1 5500/38329
batch 5 9200

batch 10 2300/21688
batch 5 20500/29751
batch 4 10800/30976
batch 0 8600/42783
batch 5 20700/29751
batch 5 20800/29751
batch 2 11300/37802
batch 3 8800/34109
batch 5 20900/29751
batch 10 2400/21688
batch 4 10900/30976
batch 5 21000/29751
batch 5 21100/29751
batch 2 11400/37802
batch 5 21300/29751
batch 1 10500/38329
batch 0 8800/42783
batch 10 2500/21688
batch 2 11500/37802
batch 5 21500/29751
batch 5 21800/29751
batch 2 11600/37802
batch 10 2600/21688
batch 1 10700/38329
batch 5 22200/29751
batch 2 11700/37802
batch 5 22300/29751
batch 4 11300/30976
batch 5 22400/29751
batch 10 2700/21688
batch 5 22600/29751
batch 2 11800/37802
batch 3 9200/34109
batch 1 10900/38329
batch 4 11400/30976
batch 5 23100/29751
batch 0 9100/42783
batch 5 23200/29751
batch 1 11000/38329
batch 4 11500/30976
batch 5 23300/29751
batch 2 12000/37802
batch 1 11100/38329
batch 10 2900/21688
batch 4 11600/30976
batch 0 9200/42783
batch 5 23800/29751
batch 5 23900/29751
batch 2 12100/37802
batch 5 24000/29751
batch 

batch 2 21500/37802
batch 4 22900/30976
batch 4 23000/30976
batch 0 15500/42783
batch 10 10700/21688
batch 2 21600/37802
batch 4 23100/30976
batch 3 19100/34109
batch 4 23200/30976
batch 2 21700/37802
batch 10 10800/21688
batch 0 15600/42783
batch 4 23300/30976
batch 2 21800/37802
batch 4 23500/30976
batch 1 22300/38329
batch 2 21900/37802
batch 4 23600/30976
batch 10 11000/21688
batch 1 22400/38329
batch 2 22000/37802
batch 0 15800/42783
batch 1 22600/38329
batch 4 24000/30976
batch 2 22200/37802
batch 0 15900/42783
batch 4 24100/30976
batch 4 24200/30976
batch 10 11300/21688
batch 2 22300/37802
batch 4 24300/30976
batch 4 24400/30976
batch 10 11400/21688
batch 2 22400/37802
batch 1 22900/38329
batch 4 24500/30976
batch 4 24600/30976
batch 2 22500/37802
batch 10 11500/21688
batch 4 24700/30976
batch 0 16100/42783
batch 4 24800/30976
batch 10 11600/21688
batch 4 24900/30976
batch 4 25000/30976
batch 10 11700/21688
batch 4 25100/30976
batch 2 22800/37802
batch 10 11800/21688
batch 4 253

batch 10 20400/21688
batch 10 20500/21688
batch 10 20600/21688
batch 10 20700/21688
batch 13 100/16773
batch 0 24500/42783
batch 16 0/12496
batch 10 21000/21688
batch 10 21200/21688
batch 10 21300/21688
batch 10 21500/21688
batch 10 21600/21688
batch 17 0/10129
batch 15 800/15703
batch 14 100/15789
batch 15 900/15703
batch 11 100/21610
batch 13 200/16773
batch 17 100/10129
batch 17 200/10129
batch 0 24700/42783
batch 13 300/16773
batch 14 200/15789
batch 17 300/10129
batch 15 2800/15703
batch 0 24800/42783
batch 17 400/10129
batch 13 400/16773
batch 11 200/21610
batch 17 500/10129
batch 14 300/15789
batch 13 500/16773
batch 17 600/10129
batch 15 5000/15703
batch 12 100/21563
batch 17 700/10129
batch 13 600/16773
batch 14 400/15789
batch 15 6300/15703
batch 15 6500/15703
batch 11 300/21610
batch 17 1000/10129
batch 15 9900/15703
batch 15 10000/15703
batch 17 1100/10129
batch 13 800/16773
batch 15 11300/15703
batch 15 11500/15703
batch 15 12200/15703
batch 15 12300/15703
batch 17 1200/10

batch 0 37100/42783
batch 13 15000/16773
batch 13 15100/16773
batch 13 15300/16773
batch 13 15400/16773
batch 13 15600/16773
batch 0 37300/42783
batch 13 15700/16773
batch 13 15900/16773
batch 13 16000/16773
batch 13 16100/16773
batch 13 16200/16773
batch 13 16300/16773
batch 13 16400/16773
batch 13 16500/16773
batch 13 16700/16773
batch 0 37600/42783
batch 0 37800/42783
batch 11 3900/21610
batch 0 38000/42783
batch 0 38100/42783
batch 0 38200/42783
batch 12 1700/21563
batch 0 38300/42783
batch 0 38500/42783
batch 11 4000/21610
batch 0 38600/42783
batch 0 38800/42783
batch 16 10100/12496
batch 0 38900/42783
batch 0 39300/42783
batch 0 39400/42783
batch 11 4100/21610
batch 0 39500/42783
batch 16 11000/12496
batch 0 39800/42783
batch 0 39900/42783
batch 0 40400/42783
batch 0 40500/42783
batch 12 1800/21563
batch 11 4200/21610
batch 0 40700/42783
batch 0 40800/42783
batch 0 40900/42783
batch 0 41000/42783
batch 0 41300/42783
batch 0 41500/42783
batch 0 41800/42783
batch 0 42000/42783
batc

In [ ]:
regex_list = regex_domain.domain_regex_extract(
    input_file_path = "../../data/EXP_domain/cluster_distance.json",
    output_file_path = "../../data/EXP_domain/regex_raw.txt",
    cluster_size_thresh=6)

In [15]:
df = regex_domain.domain_regex_check(
    input_file_path          = "../../data/EXP_domain/regex_raw.txt", 
    test_benign_file_path    = "../../data/sangfor/safe_0419.csv",
    test_malicious_file_path = "../../data/EXP_domain/malware.csv",
    result_file_path         = "../../data/EXP_domain/regex_result.txt", 
    n_jobs = 8)

batch index 0	sample index 0	FP(url) 0	FP(domain) 0	TP(url) 14	TP(domain) 14	^\w{5,6}\.wigme\.u487s\.download$
batch index 0	sample index 1	FP(url) 0	FP(domain) 0	TP(url) 4	TP(domain) 4	^(:?cs|pflycz|cnxa4g|92mqh2)\.(:?44799|awcjh8|5867|74526)\.1f5wy\.www\.8ae6q\.download$
batch index 1	sample index 0	FP(url) 0	FP(domain) 0	TP(url) 4	TP(domain) 4	^(:?k5fgu5|96889|soft|t5bxee)\.(:?9032|rtt0b7)\.lz2rb\.www\.eieois\.win$
batch index 0	sample index 2	FP(url) 0	FP(domain) 0	TP(url) 11	TP(domain) 11	^\w{5,6}\.(:?929|www|ftt)\.684io\.www\.lhc3z3\.win$
batch index 1	sample index 1	FP(url) 0	FP(domain) 0	TP(url) 66	TP(domain) 30	^\w{3,6}\.zqlkjx\.com$
batch index 0	sample index 3	FP(url) 0	FP(domain) 0	TP(url) 3	TP(domain) 3	^(:?zpyna|soft|rencai)\.(:?79739|6728)\.8u9c1\.www\.fgfgj\.win$
batch index 1	sample index 2	FP(url) 0	FP(domain) 0	TP(url) 2	TP(domain) 2	^(:?cs|9g8dzm)\.97645\.ajmv3x\.15525\.www\.gtiuo\.win$
batch index 0	sample index 4	FP(url) 0	FP(domain) 0	TP(url) 38	TP(domain) 38	^\w

batch index 2	sample index 12	FP(url) 0	FP(domain) 0	TP(url) 17	TP(domain) 17	^\w{4,6}\.d2bpv\.www\.ar9r6\.download$
batch index 7	sample index 0	FP(url) 0	FP(domain) 0	TP(url) 24	TP(domain) 14	^\w{6}\.7u44w\.tmytz\.download$
batch index 5	sample index 5	FP(url) 0	FP(domain) 0	TP(url) 52	TP(domain) 49	^\w{3,6}\.gdxxvw\.www\.josid\.faith$
batch index 3	sample index 10	FP(url) 0	FP(domain) 0	TP(url) 6	TP(domain) 6	^\w{6}\.tmsqs\.www\.wg5li\.download$
batch index 6	sample index 3	FP(url) 0	FP(domain) 0	TP(url) 4	TP(domain) 4	^(:?69189|3ns06j|pqt6vu|yq5ddp)\.xvzh0a\.j514w\.www\.73jox\.download$
batch index 1	sample index 15	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^ready4maintainsysforpcandmac\.bid$
batch index 4	sample index 8	FP(url) 0	FP(domain) 0	TP(url) 52	TP(domain) 1	^6244\.phkbg5\.wwdfpd\.37089\.3chv68\.gias81\.4ir8yy\.76452\.exea41\.dfahyp\.edu\.cn\.lchongfu\.com$
batch index 0	sample index 17	FP(url) 0	FP(domain) 0	TP(url) 20	TP(domain) 20	^\w{5,6}\.9vrtf\.k2jis\.download$
b

batch index 4	sample index 16	FP(url) 0	FP(domain) 0	TP(url) 17	TP(domain) 16	^\w{5,6}\.ssiya\.20s06\.download$
batch index 3	sample index 18	FP(url) 0	FP(domain) 0	TP(url) 17	TP(domain) 10	^\w{5,6}\.t51l3\.www\.f8d9\.faith$
batch index 1	sample index 23	FP(url) 0	FP(domain) 0	TP(url) 2	TP(domain) 2	^(:?zyzjwr|0vngqs)\.(:?43383|go09az)\.blog\.7fk4w3\.15525\.www\.gtiuo\.win$
batch index 0	sample index 25	FP(url) 0	FP(domain) 0	TP(url) 4	TP(domain) 4	^(:?54888|25007|vbs02|qq2uzh)\.(:?64720|44yva6)\.z0hca\.www\.7snyj\.download$
batch index 2	sample index 21	FP(url) 0	FP(domain) 0	TP(url) 8	TP(domain) 8	^\w{5,6}\.saui2\.bnxdg\.download$
batch index 6	sample index 12	FP(url) 0	FP(domain) 0	TP(url) 25	TP(domain) 24	^\w{4,6}\.vb1h9\.zp2h2\.download$
batch index 5	sample index 14	FP(url) 0	FP(domain) 0	TP(url) 3	TP(domain) 3	^thegreatservice2update\.(:?club|win|bid)$
batch index 7	sample index 9	FP(url) 0	FP(domain) 0	TP(url) 28	TP(domain) 28	^\w{5,6}\.f1fzn\.bhjf\.faith$
batch index 4	sample 

batch index 7	sample index 17	FP(url) 0	FP(domain) 0	TP(url) 4	TP(domain) 4	^www\.(:?goodplace-contentcentralnew|goodplace-contentcentralall)\.(:?date|bid|trade)$
batch index 1	sample index 32	FP(url) 0	FP(domain) 0	TP(url) 988	TP(domain) 988	^\d{2,5}\.(:?aibet9|aibet789)\.com$
batch index 3	sample index 27	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^thebiggerandbetterupdatesnew\.(:?win|bid)$
batch index 2	sample index 29	FP(url) 0	FP(domain) 0	TP(url) 7	TP(domain) 7	^\w{5,6}\.0r20s\.www\.04p8c\.download$
batch index 0	sample index 34	FP(url) 0	FP(domain) 0	TP(url) 215	TP(domain) 215	^\w{6,10}\.01f9y1y87\.com$
batch index 6	sample index 21	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^sourcetoupgrade\.date$
batch index 5	sample index 23	FP(url) 0	FP(domain) 0	TP(url) 42	TP(domain) 38	^\w{5,6}\.nft5f\.ljbjm\.download$
batch index 4	sample index 26	FP(url) 0	FP(domain) 0	TP(url) 109	TP(domain) 53	^\w{4,5}\.co\.cc$
batch index 7	sample index 18	FP(url) 0	FP(domain) 0	TP(url) 5	TP(doma

batch index 6	sample index 29	FP(url) 0	FP(domain) 0	TP(url) 1071	TP(domain) 1068	^13\d{2,8}\.syaniu\.com$
batch index 4	sample index 34	FP(url) 0	FP(domain) 0	TP(url) 0	TP(domain) 0	^113\.10\.184\.231$
batch index 5	sample index 31	FP(url) 0	FP(domain) 0	TP(url) 115	TP(domain) 67	^\w{5,6}\.(:?7d7ph|7p733)\.dichanpinglun\.win$
batch index 3	sample index 36	FP(url) 0	FP(domain) 0	TP(url) 83	TP(domain) 64	^\w{4,6}\.\w{3,6}\.64ryqf\.9s40r\.www\.vird\.win$
batch index 7	sample index 26	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^thebiggestofferupdate\.(:?download|review)$
batch index 1	sample index 41	FP(url) 0	FP(domain) 0	TP(url) 4	TP(domain) 3	^\w{5,6}\.73754\.66304\.uul0jd\.dllm3\.www\.q3537\.download$
batch index 0	sample index 43	FP(url) 0	FP(domain) 0	TP(url) 38	TP(domain) 37	^\w{5,6}\.(:?aqm2w|a20ew)\.k8pnq\.download$
batch index 2	sample index 38	FP(url) 0	FP(domain) 0	TP(url) 13	TP(domain) 11	^\w{5,6}\.5775x\.www\.f7i3\.win$
batch index 6	sample index 30	FP(url) 0	FP(domain) 0

...

batch index 0	sample index 675	FP(url) 0	FP(domain) 0	TP(url) 2	TP(domain) 2	^(:?alwaysorundupsysupdates|alwaysorundupsysupdate)\.download$
batch index 7	sample index 665	FP(url) 0	FP(domain) 0	TP(url) 14	TP(domain) 11	^\d{5}\.7hdns\.com$
batch index 1	sample index 682	FP(url) 0	FP(domain) 0	TP(url) 78	TP(domain) 77	^\w{6,10}\.5tbp1gvkrx\.com$
batch index 5	sample index 673	FP(url) 0	FP(domain) 0	TP(url) 14	TP(domain) 14	^\w{5,6}\.o8g4s\.gjoie\.faith$
batch index 4	sample index 682	FP(url) 0	FP(domain) 0	TP(url) 30	TP(domain) 27	^\w{5,6}\.91f1b\.cnkdjx\.win$
batch index 2	sample index 661	FP(url) 0	FP(domain) 0	TP(url) 178	TP(domain) 91	^\w{4,6}\.\w{2,6}\.x4je0d\.otov25\.76470\.66304\.uul0jd\.dllm3\.www\.q3537\.download$
batch index 6	sample index 677	FP(url) 0	FP(domain) 0	TP(url) 8	TP(domain) 8	^\w{5,6}\.(:?v7ecz1|79796)\.684io\.www\.lhc3z3\.win$
batch index 3	sample index 672	FP(url) 0	FP(domain) 0	TP(url) 11	TP(domain) 11	^\w{6}\.8keco\.q5bor\.download$
batch index 0	sample index 6

batch index 6	sample index 685	FP(url) 0	FP(domain) 0	TP(url) 22	TP(domain) 22	^\w{3,6}\.8u2e4\.vysu\.win$
batch index 7	sample index 674	FP(url) 2	FP(domain) 2	TP(url) 8	TP(domain) 8	^down0\d{7}\.cdnxiazai\.com$
batch index 2	sample index 669	FP(url) 0	FP(domain) 0	TP(url) 2	TP(domain) 2	^clickforbestandprepared2upgrades\.(:?info|bid|pro)$
batch index 4	sample index 691	FP(url) 0	FP(domain) 0	TP(url) 6	TP(domain) 6	^\w{4,6}\.blog\.5wsv86\.15525\.www\.gtiuo\.win$
batch index 0	sample index 684	FP(url) 0	FP(domain) 0	TP(url) 7	TP(domain) 7	^\w{6}\.\w{6}\.62469\.6ndldt\.www\.478sk\.win$
batch index 3	sample index 680	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^bumbs9\.oh5b4h\.w2brek\.hckv3\.www\.c1rdu\.download$
batch index 1	sample index 691	FP(url) 0	FP(domain) 0	TP(url) 9	TP(domain) 9	^\w{5,6}\.(:?z1838h|81448)\.0fdgaz\.gif26i\.fmirir\.win$
batch index 5	sample index 682	FP(url) 0	FP(domain) 0	TP(url) 14	TP(domain) 13	^\w{4,6}\.w60wi\.vnhu\.faith$
batch index 6	sample index 686	FP(

batch index 3	sample index 688	FP(url) 0	FP(domain) 0	TP(url) 61	TP(domain) 53	^\w{6,8}\.740wpd\.com$
batch index 0	sample index 692	FP(url) 0	FP(domain) 0	TP(url) 9	TP(domain) 9	^\w{5,6}\.8kiie\.478430\.win$
batch index 2	sample index 677	FP(url) 0	FP(domain) 0	TP(url) 22	TP(domain) 22	^\w{6,8}\.iey322upxw\.com$
batch index 6	sample index 694	FP(url) 0	FP(domain) 0	TP(url) 13	TP(domain) 12	^\w{6,7}\.6k1u0djrgw\.com$
batch index 4	sample index 700	FP(url) 0	FP(domain) 0	TP(url) 16	TP(domain) 7	^\w{2,4}\.464ct\.com$
batch index 7	sample index 683	FP(url) 0	FP(domain) 0	TP(url) 2	TP(domain) 2	^\w{2,6}\.soft\.38006\.utsv6\.www\.fjkjdk\.win$
batch index 1	sample index 700	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^thegoodandalwaysforupgradingnew\.review$
batch index 5	sample index 691	FP(url) 0	FP(domain) 0	TP(url) 30	TP(domain) 30	^\w{5,6}\.frfdf\.fhfk\.win$
batch index 3	sample index 689	FP(url) 0	FP(domain) 0	TP(url) 2	TP(domain) 2	^yourtrulygreat4upgrades\.(:?club|win)$
batch index

batch index 7	sample index 698	FP(url) 0	FP(domain) 0	TP(url) 19	TP(domain) 14	^\w{4,6}\.qkacm\.hfusi\.win$
batch index 2	sample index 693	FP(url) 0	FP(domain) 0	TP(url) 16	TP(domain) 16	^\w{6}\.s0o0w\.wipss\.win$
batch index 7	sample index 699	FP(url) 0	FP(domain) 0	TP(url) 28	TP(domain) 28	^\w{5,6}\.6gi8k\.vjie\.faith$
batch index 2	sample index 694	FP(url) 0	FP(domain) 0	TP(url) 18	TP(domain) 16	^\w{5,6}\.o2w0e\.gjkf67\.win$
batch index 7	sample index 700	FP(url) 0	FP(domain) 0	TP(url) 31	TP(domain) 26	^\w{5,6}\.6ism6\.djkd\.faith$
batch index 2	sample index 695	FP(url) 0	FP(domain) 0	TP(url) 6	TP(domain) 6	^\w{4,6}\.52nzi0\.hfusi\.win$
batch index 7	sample index 701	FP(url) 0	FP(domain) 0	TP(url) 28	TP(domain) 25	^\w{5,6}\.ugmu6\.fiei3\.win$
batch index 2	sample index 696	FP(url) 0	FP(domain) 0	TP(url) 26	TP(domain) 20	^\w{5,6}\.gyy24\.vtygs\.win$
batch index 7	sample index 702	FP(url) 542	FP(domain) 212	TP(url) 692	TP(domain) 159	^[A-Za-z]{3,5}\.com\.(:?co|cn)$
batch index 2	sampl

2018-04-24 22:48:03 file_io.py [line:249] DEBUG	OLD DATA FIND! REMOVING	../../data/EXP_domain/regex_result.txt
2018-04-24 22:48:03 file_io.py [line:255] DEBUG	Check Result has been dump	../../data/EXP_domain/regex_result.txt


In [16]:
df = regex_domain.domain_regex_publish(
                                  "../../data/EXP_domain/regex_result.txt",
                                  "../../data/EXP_domain/regex_publish.txt",
                                  publish_score_thresh = 0.4,
                                  publish_ratio = 10,
                                  publish_fp_thresh = 5000,
                                  publish_tp_thresh = 0)

2018-04-24 22:48:03 file_io.py [line:276] DEBUG	check Data has been loaded	../../data/EXP_domain/regex_result.txt
2018-04-24 22:48:20 regex_domain.py [line:460] DEBUG	regular expression publish	5573
2018-04-24 22:48:20 file_io.py [line:207] DEBUG	OLD DATA FIND! REMOVING	../../data/EXP_domain/regex_publish.txt
2018-04-24 22:48:20 file_io.py [line:212] DEBUG	Regex has been dump	../../data/EXP_domain/regex_publish.txt


In [3]:
predict_malicious, predict_dict, predict_dict_detail = regex_domain.malicious_domain_predict(
                      input_file_path = url_2,
                      regex_file_path = "../../data/EXP_domain/regex_publish.txt")

2018-04-25 10:06:23 file_io.py [line:191] DEBUG	Regex Data has been loaded	../../data/EXP_domain/regex_publish.txt


batch index 0	sample index 0	hit domain 0	^\w{5,6}\.wigme\.u487s\.download$
batch index 0	sample index 1	hit domain 0	^(:?cs|pflycz|cnxa4g|92mqh2)\.(:?44799|awcjh8|5867|74526)\.1f5wy\.www\.8ae6q\.download$
batch index 0	sample index 2	hit domain 0	^\w{5,6}\.(:?929|www|ftt)\.684io\.www\.lhc3z3\.win$
batch index 0	sample index 3	hit domain 0	^(:?zpyna|soft|rencai)\.(:?79739|6728)\.8u9c1\.www\.fgfgj\.win$
batch index 0	sample index 4	hit domain 0	^\w{6,8}\.r0s74gtbx\.com$
batch index 0	sample index 5	hit domain 0	^\w{4,6}\.mmhtz\.download$
batch index 0	sample index 6	hit domain 0	^\d{4,5}\.\w{6}\.onysk\.www\.ngtuh\.win$
batch index 0	sample index 7	hit domain 0	^\w{4,6}\.hi5rht\.15525\.www\.gtiuo\.win$
batch index 1	sample index 0	hit domain 0	^\w{3,6}\.zqlkjx\.com$
batch index 0	sample index 8	hit domain 0	^\w{5,6}\.9bclw\.www\.lwzbf\.download$
batch index 1	sample index 1	hit domain 0	^(:?cs|9g8dzm)\.97645\.ajmv3x\.15525\.www\.gtiuo\.win$
batch index 0	sample index 9	hit domain 0	^[A-Z

batch index 4	sample index 5	hit domain 0	^\w{2,6}\.u4e271\.sdkdj\.win$
batch index 0	sample index 35	hit domain 0	^facebook\.com\.profile\.accounts\.login\.userid41d0\w{8,16}\.(:?4ws5|2ms5)\.com$
batch index 1	sample index 28	hit domain 0	^down\d{4,5}\.yzzzn\.com$
batch index 3	sample index 11	hit domain 0	^(:?216|210)\.\d{2,3}\.184\.10$
batch index 0	sample index 36	hit domain 0	^38977\.ydnet7\.96994\.2o8ctw\.jhw5q\.www\.8ae6q\.download$
batch index 2	sample index 19	hit domain 0	^[A-Za-z]{7,12}\.sube-halkbankasitr\.com$
batch index 1	sample index 29	hit domain 0	^\w{5,6}\.k5gr4k\.cdnpp\.www\.k8pnq\.download$
batch index 4	sample index 6	hit domain 0	^\w{2,3}\.gpxz\.net$
batch index 1	sample index 30	hit domain 0	^\w{6}\.qv4zh\.www\.fkjjd\.win$
batch index 0	sample index 37	hit domain 0	^\w{3,6}\.p0e4kq\.4206e\.fmirir\.win$
batch index 3	sample index 12	hit domain 0	^\w{5,6}\.aa2ku\.uqwo\.win$
batch index 4	sample index 7	hit domain 0	^www\.(:?yourbestcornerforcontentingup|yourbestco

batch index 0	sample index 49	hit domain 0	^\w{6}\.cp717\.www\.wejfi\.win$
batch index 1	sample index 40	hit domain 0	^\w{5,6}\.73754\.66304\.uul0jd\.dllm3\.www\.q3537\.download$
batch index 3	sample index 30	hit domain 0	^\w{3,6}\.www\.antwa\.download$
batch index 7	sample index 0	hit domain 0	^\w{5,6}\.v32s4\.www\.ngtuh\.win$
batch index 4	sample index 26	hit domain 0	^113\.10\.184\.231$
batch index 2	sample index 34	hit domain 0	^(:?theonlygoodplaceforcontentnow|theonlygoodplacetocontents)\.(:?win|bid|trade|date)$
batch index 1	sample index 41	hit domain 0	^\w{5,6}\.a0g6q\.ea8nw\.download$
batch index 7	sample index 1	hit domain 0	^www\.jlrcreations\.com$
batch index 5	sample index 6	hit domain 0	^\w{5,6}\.wu66s\.jkdf\.faith$
batch index 4	sample index 27	hit domain 0	^\w{4,6}\.ar9r6\.download$
batch index 6	sample index 6	hit domain 0	^\w{4,6}\.i0sxjj\.15525\.www\.gtiuo\.win$
batch index 0	sample index 50	hit domain 0	^\w{3,6}\.(:?56280|x9iicb)\.bq86i\.www\.ndur3\.download$
batch i

batch index 7	sample index 13	hit domain 0	^www\.(:?goodplace-contentcentralnew|goodplace-contentcentralall)\.(:?date|bid|trade)$
batch index 1	sample index 52	hit domain 0	^www\.[A-Za-z]{4,7}\.com\.(:?au|uy)$
batch index 4	sample index 38	hit domain 0	^\w{5,6}\.26g64\.cjkf\.win$
batch index 2	sample index 45	hit domain 0	^\w{6,8}\.ivdhhwor9\.com$
batch index 5	sample index 21	hit domain 0	^\w{5,6}\.(:?7d7ph|7p733)\.dichanpinglun\.win$
batch index 6	sample index 20	hit domain 0	^13\d{2,8}\.syaniu\.com$
batch index 0	sample index 64	hit domain 0	^(:?safeandorganized2updating|safeandorganizedupdating)\.(:?club|win)$
batch index 7	sample index 14	hit domain 0	^\w{2,6}\.(:?zjpaxj|zpyna)\.6xxm1z\.lrbtb\.fjifi\.win$
batch index 1	sample index 53	hit domain 0	^\w{6}\.48z8tz\.b5jts5\.a2yud3\.17393\.76452\.exea41\.dfahyp\.edu\.cn\.lchongfu\.com$
batch index 5	sample index 22	hit domain 0	^nysvp\.34473\.www\.wuus\.win$
batch index 4	sample index 39	hit domain 0	^\w{5,6}\.b89qz\.www\.josid\.faith

...

batch index 7	sample index 650	hit domain 0	^\w{5,6}\.54513\.oephp\.www\.fh839\.win$
batch index 3	sample index 651	hit domain 0	^2p-0\.(:?19-20000000|19-40000000)\.f00\.170c\.(:?22a2|22a1)\.2f4a\.800\.0\.\w{26}\.avts\.mcafee\.com$
batch index 6	sample index 646	hit domain 0	^\w{5,6}\.lsqtp\.www\.bgsk\.win$
batch index 0	sample index 652	hit domain 0	^(:?perfectcodeforfreeupgrades|perfectcodeforfreeupdates)\.(:?pw|pro)$
batch index 1	sample index 651	hit domain 0	^\w{5,6}\.0767c4\.www\.zjo1b\.download$
batch index 4	sample index 635	hit domain 0	^\w{5,6}\.5w0w2\.www\.djfij\.win$
batch index 5	sample index 646	hit domain 0	^\w{6}\.www\.7guy5\.download$
batch index 2	sample index 594	hit domain 0	^betterandfree2upgrading\.(:?date|review)$
batch index 6	sample index 647	hit domain 0	^aapache\.blogspot\.(:?ca|com)$
batch index 3	sample index 652	hit domain 0	^(:?perfectcenter4newsoftwarepcandmac|perfect-centerfornewsoftwarepcandmac|perfect-center4newsoftwarepcandmac)\.(:?pw|download)$
batc

batch index 6	sample index 663	hit domain 0	^\w{5,6}\.8wug2\.fiir\.win$
batch index 0	sample index 662	hit domain 0	^\w{6}\.www\.ea8nw\.download$
batch index 4	sample index 649	hit domain 0	^gay-porn-pictures\.blogspot\.com$
batch index 3	sample index 670	hit domain 0	^\w{5,6}\.y0swe\.7guy5\.download$
batch index 2	sample index 600	hit domain 0	^(:?3zg8u3|23107|7057kx)\.cs\.n59dj\.dioe\.win$
batch index 0	sample index 663	hit domain 0	^\w{6}\.q8x95\.www\.vjhg\.faith$
batch index 1	sample index 659	hit domain 0	^\w{6}\.aeu62\.757858\.win$
batch index 6	sample index 664	hit domain 0	^\w{5,6}\.me8ui\.fjkjf\.win$
batch index 5	sample index 655	hit domain 0	^\w{4,6}\.www\.geie\.faith$
batch index 3	sample index 671	hit domain 0	^\w{5,6}\.\w{4,6}\.4szeoe\.ajy2xe\.uul0jd\.dllm3\.www\.q3537\.download$
batch index 4	sample index 650	hit domain 0	^moyu\.(:?16324|13086)\.dqlrlb\.wkswg\.fjit4\.win$
batch index 6	sample index 665	hit domain 0	^\w{5,6}\.www\.fgi5\.win$
batch index 5	sample index 656

batch index 4	sample index 663	hit domain 0	^\w{5,6}\.vd354\.www\.f4k5\.faith$
batch index 3	sample index 682	hit domain 0	^\w{4,6}\.1u7l9i\.0fdgaz\.gif26i\.fmirir\.win$
batch index 0	sample index 676	hit domain 0	^\w{5,6}\.vtb5d\.3s0lp\.download$
batch index 5	sample index 668	hit domain 0	^(:?dgjjhw|3w4kge|j431yx)\.(:?6b2lnl|12856)\.l507k\.www\.3s0lp\.download$
batch index 6	sample index 680	hit domain 0	^\w{6}\.nlbdd\.www\.ajioew\.win$
batch index 4	sample index 664	hit domain 0	^000\w{4}\.(:?wcomhost|rcomhost)\.com$
batch index 3	sample index 683	hit domain 0	^4nnolk\.68902\.yvi1y5\.ynngh\.www\.vhty\.win$
batch index 0	sample index 677	hit domain 0	^\w{5,6}\.m8xei\.www\.jkdf\.faith$
batch index 4	sample index 665	hit domain 0	^s\.15bb\.com$
batch index 5	sample index 669	hit domain 0	^\w{4,6}\.w60wi\.vnhu\.faith$
batch index 1	sample index 666	hit domain 0	^www\.(:?your-readysystems4contents|your-readysystemsforcontentall)\.(:?date|trade)$
batch index 0	sample index 678	hit domain 

batch index 7	sample index 690	hit domain 0	^\w{5,6}\.0q8c2\.lhcbp\.win$
batch index 1	sample index 678	hit domain 0	^\w{5,6}\.ouju3\.dur34\.win$
batch index 4	sample index 672	hit domain 0	^\w{5,6}\.vqlt5d\.as73yy\.i52nod\.onm1ue\.4ir8yy\.76452\.exea41\.dfahyp\.edu\.cn\.lchongfu\.com$
batch index 6	sample index 688	hit domain 0	^[^\.]{7,8}\.proastrohouse\.ru$
batch index 0	sample index 693	hit domain 0	^thegreatcentercontentsafenew\.(:?stream|win|bid|date|review)$
batch index 2	sample index 617	hit domain 0	^\w{5,6}\.4wq6s\.vjhg\.faith$
batch index 5	sample index 681	hit domain 0	^\w{5,6}\.tw6hb\.www\.rg13w\.download$
batch index 7	sample index 691	hit domain 0	^\w{4,6}\.qkacm\.hfusi\.win$
batch index 4	sample index 673	hit domain 0	^(:?bigandsolidforupdating|thebigandsolidforupdating)\.(:?review|bid)$
batch index 2	sample index 618	hit domain 0	^\w{4,6}\.0e29w\.www\.sdfnvu\.win$
batch index 1	sample index 679	hit domain 0	^\w{4,6}\.\w{5,6}\.y7hbw4\.vgzc8\.www\.vjdod\.win$
batch index

batch index 4	sample index 691	hit domain 0	^\w{3,6}\.gdxxvw\.www\.josid\.faith$
batch index 2	sample index 639	hit domain 0	^(:?www|24pccheck)\.the-readysystems4contents\.download$
batch index 4	sample index 692	hit domain 0	^\w{5,6}\.9serj\.www\.ngtuh\.win$
batch index 2	sample index 640	hit domain 0	^rencai\.\w{5,6}\.15525\.www\.gtiuo\.win$
batch index 4	sample index 693	hit domain 0	^\w{6}\.70rbc\.www\.li740\.download$
batch index 2	sample index 641	hit domain 0	^\w{4,6}\.94986\.kir6f\.www\.20s06\.download$
batch index 2	sample index 642	hit domain 0	^(:?eee|p7kci9|kzsit6|kvby0v)\.(:?q5t83j|x868nm|c00qn7)\.ogr2pu\.1ooej1\.51ayk\.www\.cnkdjx\.win$
batch index 4	sample index 694	hit domain 0	^\w{4,6}\.www\.hfusi\.win$
batch index 2	sample index 643	hit domain 0	^\w{5,6}\.02gsg\.gfjir\.win$
batch index 2	sample index 644	hit domain 0	^\w{5,6}\.4wrbvm\.www\.wg5li\.download$
batch index 4	sample index 695	hit domain 0	^uu3puj\.g6ab7g\.wsl8qg\.0gdind\.q81huf\.thvgl\.www\.vjde\.faith$
bat

In [4]:
len(predict_malicious)

3368

In [5]:
for k, v in predict_dict_detail.iteritems():
    print "%40s\t\tdomain:%d\turl:%d" %(k, len(v), sum([len(_) for _ in v]))

                 ^\w{4,5}\.ebutscdo\.cn$		domain:12	url:13
      ^\d{5}\.url\.(:?789msw|7wkw)\.com$		domain:25	url:43
               ^bd\d{1,2}\.52lishi\.com$		domain:1	url:2
             ^\d{4,5}\.url\.246546\.com$		domain:116	url:289
             ^\d{4,5}\.url\.tudown\.com$		domain:1	url:1
               ^\d{4,5}\.url\.7wkw\.com$		domain:7	url:8
          ^\w{3,15}\.000webhostapp\.com$		domain:1	url:1
                   ^\w{2,4}\.mqego\.com$		domain:12	url:52
               ^\d{5}\.xc\.gongnou\.com$		domain:245	url:1807
          ^\d{4,5}\.url\.9xiazaiqi\.com$		domain:29	url:50
                ^\w{5}\.chinayijia\.net$		domain:2	url:2
         ^[A-Za-z]{3,5}\.com\.(:?co|cn)$		domain:1	url:1
              ^\d{4,5}\.url\.222bz\.com$		domain:36	url:47
                 ^\d{6}\.sdyintong\.com$		domain:49	url:49
             ^mitaosp\d{3}\.17arch\.com$		domain:5	url:8
               ^\d{5}\.xc\.cangpie\.com$		domain:212	url:998
    ^[A-Za-z]{5,12}\.000webhostapp\.com$		domain:1	url:1
      

In [29]:
safe_domain_forest = jl.load("../../data/url_forest/safe_domain_forest_mod.jl.z")
mal_domain_forest  = jl.load("../../data/url_forest/mal_domain_forest.jl.z")

In [32]:
domain_reputation_thresh = 0.1
df = url_forest.analyze_domain_batch(predict_malicious, safe_domain_forest, mal_domain_forest, verbose = False)
df_publish = df.loc[df.domain_reputation>domain_reputation_thresh]
predict_malicious_final = list(df_publish.url)
print(len(predict_malicious_final))

3332


In [33]:
set(predict_malicious) - set(predict_malicious_final)

{'http://14614.url.789msw.com/down/WPS%20Office@795_3141.exe',
 'http://18954.url.789msw.com/xiaz/%E4%BD%93%E9%9D%A2%E4%BA%8E%E6%96%87%E6%96%87%E9%AB%98%E5%93%81%E8%B4%A8mp3@814_186184.exe',
 'http://18954.url.789msw.com/xiaz/%E5%8E%89%E5%AE%B3%E4%BA%86%E6%88%91%E7%9A%84%E5%9B%BD%E8%BF%85%E9%9B%B7BT%E7%A7%8D%E5%AD%90@814_192391.exe',
 'http://18954.url.789msw.com/xiaz/%E5%91%A8%E6%9D%B0%E4%BC%A6%E7%AD%89%E4%BD%A0%E4%B8%8B%E8%AF%BE%E5%BA%A6%E7%9B%98mp3@814_187404.exe',
 'http://18954.url.789msw.com/xiaz/%E5%A4%B4%E5%8F%B7%E7%8E%A9%E5%AE%B6%E9%AB%98%E6%B8%85bt%E7%A7%8D%E5%AD%90%E8%B5%84%E6%BA%90@814_193038.exe',
 'http://18954.url.789msw.com/xiaz/%E5%BD%BC%E5%BE%97%E5%85%94%E7%94%B5%E5%BD%B1%E5%AE%8C%E6%95%B4%E7%89%88%E7%99%BE%E5%BA%A6%E7%BD%91%E7%9B%98@814_190585.exe',
 'http://18954.url.789msw.com/xiaz/%E5%BD%BC%E5%BE%97%E5%85%94%E7%94%B5%E5%BD%B12018%E7%99%BE%E5%BA%A6%E4%BA%91%E4%B8%8B%E8%BD%BD@816_206172.exe',
 'http://18954.url.789msw.com/xiaz/%E6%97%A0%E7%95%8F%E7%99%BE%E5%BA%A6%E4

In [6]:
import validate
reload(validate)
from validate import ValidateWithVT

In [7]:
valid_obj = ValidateWithVT()
df = valid_obj.download_file(predict_malicious, 
                             "/home/sparrow/sparrow/data/edr_test/domain")

2018-04-25 10:06:47 validate.py [line:51] DEBUG	index	0	http://19622.xc.gongnou.com	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-25 10:06:48 validate.py [line:51] DEBUG	index	1	http://19959.xc.cangpie.com/xiaz/%E5%85%84%E5%BC%9F%E6%BF%80%E5%85%89MFC-7360@311_12607.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-25 10:06:48 validate.py [line:51] DEBUG	index	7	http://20102.xc.gongnou.com/xiaz/%E7%94%9F%E9%97%A8%E7%99%BE%E5%BA%A6%E4%BA%91%E4%B8%8B%E8%BD%BD@407_2.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-25 10:06:48 validate.py [line:51] DEBUG	index	5	http://20051.xc.gongnou.com/xiaz/%E7%A0%B4%E4%BA%A7%E5%A7%90%E5%A6%B9%E7%AC%AC%E4%B8%80%E5%AD%A3%E5%9C%A8%E7%BA%BF%E7%9C%8B@610_2.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-25 10:06:48 validate.py [line:51] DEBUG	index	3	http://19939.url.246546.com/xiaz/MatlabR2017a64%E4%BD%8D%E6%B1%89%E5%8C%96%E6%B3%A8%E5%86%8C%E7%89%

2018-04-25 10:06:52 validate.py [line:51] DEBUG	index	34	http://19906.xc.gongnou.com/xiaz/%E7%BA%A2%E6%B5%B7%E8%A1%8C%E5%8A%A8%E7%99%BE%E5%BA%A6%E4%BA%91@407_2.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-25 10:06:52 validate.py [line:51] DEBUG	index	36	http://19904.xc.cangpie.com	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-25 10:06:52 validate.py [line:51] DEBUG	index	35	http://19938.xc.gongnou.com/xiaz/%E8%9D%99%E8%9D%A0%E4%BE%A0%E5%89%8D%E4%BC%A02%E9%BB%91%E6%9A%97%E9%AA%91%E5%A3%AB%E9%AB%98%E6%B8%85_%E7%99%BE%E5%BA%A6%E4%BA%91%E7%BD%91%E7%9B%98_%E5%9C%A8%E7%BA%BF%E8%A7%82%E7%9C%8B_%E8%BF%85%E9%9B%B7%E4%B8%8B%E8%BD%BD@439_2.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-25 10:06:52 validate.py [line:51] DEBUG	index	40	http://20109.xc.gongnou.com/xiaz/AbCd%E7%BB%83%E6%89%93%E5%AD%97V1.4.2%E5%85%8D%E8%B4%B9%E7%89%88@240_221481.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff


2018-04-25 10:06:56 validate.py [line:51] DEBUG	index	71	http://19930.xc.gongnou.com/xiaz/LLshishicai-v5.41@278_67112.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-25 10:06:56 validate.py [line:51] DEBUG	index	66	http://19955.xc.cangpie.com/xiaz/Office2010@394_2.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-25 10:06:56 validate.py [line:51] DEBUG	index	75	http://19939.xc.gongnou.com/xiaz/%E8%80%81%E7%94%B7%E5%AD%A9%E7%94%B5%E8%A7%86%E5%89%A7%E5%85%A8%E9%9B%86%E5%85%8D%E8%B4%B9%E5%9C%A8%E7%BA%BF%E8%A7%82%E7%9C%8B%E8%80%81%E7%94%B7%E5%AD%A9%E7%94%B5%E8%A7%86%E5%89%A7%E5%85%A8%E9%9B%861-46%E7%99%BE%E5%BA%A6%E4%BA%91%E7%BD%91%E7%9B%98%E8%B5%84%E6%BA%90@1241_2.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-25 10:06:56 validate.py [line:51] DEBUG	index	67	http://19721.xc.gongnou.com/xiaz/Autodesk3dsMax2016%E7%AE%80%E4%BD%93%E4%B8%AD%E6%96%87%E7%89%88(%E9%99%84%E6%B3%A8%E5%86%8C%E6%9C%BA)@284_265275.ex

2018-04-25 10:07:00 validate.py [line:51] DEBUG	index	93	http://14614.xc.gongnou.com/down/Adobe%20Reader%20XI%20PDF%E9%98%85%E8%AF%BB%E5%99%A8@626_3116.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-25 10:07:00 validate.py [line:51] DEBUG	index	100	http://19715.xc.cangpie.com/down/SeewoLink%E5%B8%8C%E6%B2%83%E6%8E%88%E8%AF%BE%E5%8A%A9%E6%89%8B%E7%94%B5%E8%84%91%E7%89%883.0.12.2956%E5%AE%98%E6%96%B9%E5%85%8D%E8%B4%B9%E7%89%88@158_185355.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-25 10:07:00 validate.py [line:51] DEBUG	index	105	http://19548.xc.cangpie.com/down/Autocad2014%E7%A0%B4%E8%A7%A3%E7%89%88%E7%AE%80%E4%BD%93%E4%B8%AD%E6%96%87%E7%89%88%EF%BC%8832/64%E4%BD%8D%EF%BC%89@158_77748.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-25 10:07:00 validate.py [line:51] DEBUG	index	103	http://20057.xc.gongnou.com/xiaz/%E6%9A%B4%E8%A3%82%E6%97%A0%E5%A3%B0@407_2.exe	3954ed23ba188e657959509fd418273f83f7f

2018-04-25 10:07:04 validate.py [line:51] DEBUG	index	118	http://20060.xc.cangpie.com/xiaz/Boom@212_147826.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-25 10:07:04 validate.py [line:51] DEBUG	index	135	http://c1.mqego.com/soft3/qs.zip	b48d95ac62a854bef5fa759e42335557c03a48c9808362d1a78f76b1d3b7c614
2018-04-25 10:07:04 validate.py [line:51] DEBUG	index	128	http://20105.xc.gongnou.com/xiaz/EpsonL1300@311_16960.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-25 10:07:05 validate.py [line:51] DEBUG	index	136	http://19d61.ebutscdo.cn:92/kf105829v/MicrosoftOffice2016%E7%A0%B4%E8%A7%A3%E7%89%88%E5%AE%98%E6%96%B9%E4%B8%AD%E6%96%87%E6%89%B9%E9%87%8F%E6%8E%88%E6%9D%83%E7%89%883264%E4%BD%8D%4054_26	56ed482bf50ab5eca9ed5debd5e34e9cdb180bb2f05e99ddb68ec0adc03b4c92
2018-04-25 10:07:05 validate.py [line:51] DEBUG	index	134	http://19933.xc.cangpie.com/xiaz/%E5%A4%A9%E7%94%9F%E4%B8%80%E5%AF%B9%E5%85%A8%E9%9B%86%E5%9C%A8%E7%BA%BF%E8%A7%82%E7%9

...

2018-04-25 10:13:29 validate.py [line:51] DEBUG	index	3212	http://20058.xc.gongnou.com/xiaz/%E5%80%9A%E5%A4%A9%E5%B1%A0%E9%BE%99%E8%AE%B02001%E5%9B%BD%E8%AF%AD-%E6%B8%AF%E5%89%A7%E7%BD%91@668_2.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-25 10:13:29 validate.py [line:51] DEBUG	index	3213	http://19954.url.246546.com/xiaz/%E8%8B%8F%E6%9F%93%E6%9F%93%E8%BF%BD%E5%A4%AB%E8%AE%B0_%E7%99%BE%E5%BA%A6%E7%BD%91%E7%9B%98-%E7%BD%91%E7%9B%98007@554_2.exe	d540e7934d1398f2b6b19e7b7c17e12c387866386f555b8444029ad713312e7c
2018-04-25 10:13:30 validate.py [line:51] DEBUG	index	3219	http://19959.xc.gongnou.com/xiaz/axure8.0%E5%85%8D%E8%B4%B9%E8%A7%86%E9%A2%91%E6%95%99%E7%A8%8B%E5%90%88%E9%9B%86@68_294343.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-25 10:13:30 validate.py [line:51] DEBUG	index	3215	http://19941.xc.gongnou.com/xiaz/%E5%8F%A4%E5%A2%93%E4%B8%BD%E5%BD%B1%EF%BC%9A%E6%BA%90%E8%B5%B7%E4%B9%8B%E6%88%98%E9%AB%98%E6%B8%85%E5%9C%A8%E7%

2018-04-25 10:13:32 validate.py [line:51] DEBUG	index	3249	http://19961.xc.cangpie.com	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-25 10:13:32 validate.py [line:51] DEBUG	index	3252	http://19713.xc.gongnou.com/xiaz/%E7%BA%A2%E6%B5%B7%E8%A1%8C%E5%8A%A8@407_2.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-25 10:13:33 validate.py [line:51] DEBUG	index	3251	http://19941.xc.gongnou.com/xiaz/%E5%A6%96%E9%93%83%E9%93%83%E9%AB%98%E6%B8%85%E5%9C%A8%E7%BA%BF%E8%A7%82%E7%9C%8B-%E7%94%B5%E5%BD%B1%E5%A6%96%E9%93%83%E9%93%83%E4%B8%8B%E8%BD%BD-6080%E6%96%B0%E8%A7%86%E8%A7%89%E5%BD%B1%E9%99%A2@1267_2.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-25 10:13:33 validate.py [line:51] DEBUG	index	3254	http://19268.xc.cangpie.com/xiaz/%E3%80%8ANBA2K18%E3%80%8BCODEX%E7%A0%B4%E8%A7%A3%E8%A1%A5%E4%B8%81@141_310999.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-25 10:13:33 validate.py [line:51]

2018-04-25 10:13:38 validate.py [line:51] DEBUG	index	3282	http://20100.xc.cangpie.com/xiaz/office2007%E5%AE%98%E6%96%B9%E4%B8%8B%E8%BD%BD%E5%85%8D%E8%B4%B9%E5%AE%8C%E6%95%B4%E7%89%88@418_3825.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-25 10:13:38 validate.py [line:51] DEBUG	index	3280	http://20106.xc.gongnou.com/xiaz/%E5%A5%A5%E7%89%B9%E6%9B%BC%E6%A0%BC%E6%96%97%E8%BF%9B%E5%8C%964%E5%AE%98%E6%96%B9%E6%AD%A3%E5%BC%8F%E4%B8%AD%E6%96%87%E7%89%88@160_13608.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-25 10:13:38 validate.py [line:51] DEBUG	index	3281	http://20098.xc.gongnou.com/xiaz/%E8%81%94%E6%83%B3LJ2400Pro@311_19611.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-25 10:13:38 validate.py [line:51] DEBUG	index	3272	http://19928.xc.gongnou.com/xiaz/%E5%87%AF%E7%AB%8B%E5%BE%B72015%E5%86%AC%E5%AD%A3%E7%89%88v5.1@212_51265.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-0

2018-04-25 10:13:41 validate.py [line:51] DEBUG	index	3313	http://20097.xc.cangpie.com/xiaz/91pron%E6%97%A0%E9%99%90%E5%88%B6%E5%85%8D%E8%B4%B9%E7%9C%8B@305_81363.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-25 10:13:41 validate.py [line:51] DEBUG	index	3315	http://20129.xc.gongnou.com/xiaz/%E7%BA%A2%E6%B5%B7%E8%A1%8C%E5%8A%A8HD@395_2.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-25 10:13:41 validate.py [line:51] DEBUG	index	3311	http://20075.url.246546.com/xiaz/%E5%AD%A4%E6%9C%AC%E6%98%8E%E4%BB%A3%E4%BA%BA%E7%89%A9%E5%B0%8F%E4%BC%A06.pdf%E5%AD%A4%E6%9C%AC%E6%98%8E%E4%BB%A3%E4%BA%BA%E7%89%A9%E5%B0%8F%E4%BC%A0%EF%BC%88%E5%85%A8%E5%8D%81%E5%86%8C%EF%BC%89%E8%91%A3%E5%85%89%E5%92%8C%E5%BC%A0%E5%9B%BD%E4%B9%94%E7%BC%96%E5%85%A8%E5%9B%BD%E5%9B%BE%E4%B9%A6%E9%A6%86%E6%96%87%E7%8C%AE%E7%BC%A9%E5%BE%AE%E4%B8%AD%E5%BF%83%E5%87%BA%E7%89%88,2003%E5%B9%B4_%E7%99%BE%E5%BA%A6%E7%BD%91%E7%9B%98-%E7%BD%91%E7%9B%98007@554_2.exe	d540e7934d13

2018-04-25 10:13:49 validate.py [line:51] DEBUG	index	3339	http://20025.xc.gongnou.com	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-25 10:13:49 validate.py [line:51] DEBUG	index	3346	http://19954.xc.gongnou.com/xiaz/24%E3%80%8A%E5%85%A8%E7%90%83%E6%9C%80%E5%80%BC%E5%BE%97%E5%90%AC%E7%9A%84%E8%8B%B1%E6%96%87%E6%AD%8C3CD%E3%80%8B%E7%99%BE%E5%BA%A6%E4%BA%91%E4%B8%8B%E8%BD%BD%E5%9C%B0%E5%9D%80@684_2.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-25 10:13:49 validate.py [line:51] DEBUG	index	3345	http://19958.xc.gongnou.com/xiaz/office2007%E5%AE%98%E6%96%B9%E4%B8%8B%E8%BD%BD%E5%85%8D%E8%B4%B9%E5%AE%8C%E6%95%B4%E7%89%88@394_2.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-25 10:13:49 validate.py [line:51] DEBUG	index	3344	http://20038.xc.cangpie.com/xiaz/%E8%85%BE%E8%AE%AF%E6%B8%B8%E6%88%8F%E5%B9%B3%E5%8F%B0v1.26.1.3357@212_41050.exe	3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04

In [8]:
df.to_csv("../../data/EXP_domain/df_res_url.csv", index = False)

In [10]:
sha256_list = valid_obj.upload_file_to_vt("/home/sparrow/sparrow/data/edr_test/domain")

2018-04-25 10:14:12 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com
2018-04-25 10:14:14 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "POST /vtapi/v2/file/scan?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1 HTTP/1.1" 200 580
2018-04-25 10:14:14 validate.py [line:170] DEBUG	success to upload file to VT 3954ed23ba188e657959509fd418273f83f7fa186dbda190d5284b1aa92ebbff
2018-04-25 10:14:14 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com
2018-04-25 10:14:16 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "POST /vtapi/v2/file/scan?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1 HTTP/1.1" 200 580
2018-04-25 10:14:16 validate.py [line:170] DEBUG	success to upload file to VT 3f1d5f08fa794a139a23f3d20d332905b041fe0a3484d3d35296de9a304b7f65
2018-04-25 10:14:16 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): ww

2018-04-25 10:17:31 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "POST /vtapi/v2/file/scan?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1 HTTP/1.1" 200 580
2018-04-25 10:17:31 validate.py [line:170] DEBUG	success to upload file to VT b176fa692cc681b37a311a6c111efe27b1d5c2cf431ade151e10c798c817ff1f
2018-04-25 10:17:31 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com
2018-04-25 10:17:36 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "POST /vtapi/v2/file/scan?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1 HTTP/1.1" 200 580
2018-04-25 10:17:36 validate.py [line:170] DEBUG	success to upload file to VT 77c99878691657a6ddc8a98b4dd3487da54c1cf421cbc880e4e32fc50e14fd32
2018-04-25 10:17:36 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com
2018-04-25 10:17:44 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "POST 

2018-04-25 10:20:18 validate.py [line:170] DEBUG	success to upload file to VT ada861a325af53d85f76a2f5e70d215df3e2c4dbf4a3bb7740f24d0c21a2a4e8
2018-04-25 10:20:19 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com
2018-04-25 10:20:20 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "POST /vtapi/v2/file/scan?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1 HTTP/1.1" 200 580
2018-04-25 10:20:20 validate.py [line:170] DEBUG	success to upload file to VT dad44a53f0d2d3559d49bed499cbc0674433e805590ba4962a28e15c3328ae36
2018-04-25 10:20:20 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com
2018-04-25 10:20:22 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "POST /vtapi/v2/file/scan?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1 HTTP/1.1" 200 580
2018-04-25 10:20:22 validate.py [line:170] DEBUG	success to upload file to VT ba76ef5d4e1294

2018-04-25 10:22:33 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com
2018-04-25 10:23:46 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "POST /vtapi/v2/file/scan?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1 HTTP/1.1" 200 580
2018-04-25 10:23:46 validate.py [line:170] DEBUG	success to upload file to VT 5d585333178f18b68049eec3294adb1deed85275f3b9ea31d96ac57d3ad8ecfe
2018-04-25 10:23:46 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com
2018-04-25 10:23:52 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "POST /vtapi/v2/file/scan?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1 HTTP/1.1" 200 580
2018-04-25 10:23:52 validate.py [line:170] DEBUG	success to upload file to VT 30b81bf6142418faa7277cf6efd87c0912b6c25bbe25d3aea6c746586e29b5e9
2018-04-25 10:23:52 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): ww

In [24]:
df = valid_obj.validate_file_with_vt(sha256_list)

2018-04-25 10:56:04 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com
2018-04-25 10:56:05 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "GET /vtapi/v2/file/report?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1&resource=56ed482bf50ab5eca9ed5debd5e34e9cdb180bb2f05e99ddb68ec0adc03b4c92 HTTP/1.1" 200 None
2018-04-25 10:56:05 validate.py [line:197] DEBUG	index 0	 sha256 56ed482bf50ab5eca9ed5debd5e34e9cdb180bb2f05e99ddb68ec0adc03b4c92	positives 27
2018-04-25 10:56:21 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com
2018-04-25 10:56:22 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "GET /vtapi/v2/file/report?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1&resource=763f159b69604699d8267561fde20c2a57f5c6044bf49a4a508e4a2c94fa4f2b HTTP/1.1" 200 None
2018-04-25 10:56:22 validate.py [line:197] DEBUG	index 1	 sha256 763f159b69604699d8

In [28]:
df.to_csv("../../data/EXP_domain/df_res_sha256.csv", index = False)